In [2]:
import pandas as pd
crypto_df = pd.read_csv('/Users/priyankapatil/Downloads/ETL_Dataset.csv')
crypto_df.head()

,slug,asset,name,date,ranknow,open,high,low,close,volume,market,close_ratio,spread
0,target-coin,TGT,Target Coin,29-09-2017,607,0.028961,0.054766,0.028961,0.041777,69996,0.0,0.4966,0.03
1,target-coin,TGT,Target Coin,30-09-2017,607,0.041783,0.046196,0.031435,0.031744,5725,0.0,0.0209,0.01
2,target-coin,TGT,Target Coin,01-10-2017,607,0.031761,0.035957,0.021040,0.028385,5012,0.0,0.4924,0.01
3,target-coin,TGT,Target Coin,02-10-2017,607,0.028375,0.054595,0.020417,0.022525,8010,0.0,0.0617,0.03
4,target-coin,TGT,Target Coin,03-10-2017,607,0.022527,0.032225,0.020211,0.020359,1787,0.0,0.0123,0.01


In [3]:
import numpy as np

assetsCode = ['BTC','ETH','XRP','LTC']

# coverting open, close, high and low price of crypto currencies into GBP values since current price is in Dollars
# if currency belong to this list ['BTC','ETH','XRP','LTC']
crypto_df['open'] = crypto_df[['open', 'asset']].apply(lambda x: (float(x[0]) * 0.80) if x[1] in assetsCode else np.nan, axis=1)
crypto_df['close'] = crypto_df[['close', 'asset']].apply(lambda x: (float(x[0]) * 0.80) if x[1] in assetsCode else np.nan, axis=1)
crypto_df['high'] = crypto_df[['high', 'asset']].apply(lambda x: (float(x[0]) * 0.80) if x[1] in assetsCode else np.nan, axis=1)
crypto_df['low'] = crypto_df[['low', 'asset']].apply(lambda x: (float(x[0]) * 0.80) if x[1] in assetsCode else np.nan, axis=1)

# dropping rows with null values by asset column
crypto_df.dropna(inplace=True)

# reset the data frame index
crypto_df.reset_index(drop=True ,inplace=True)
crypto_df.head()

,slug,asset,name,date,ranknow,open,high,low,close,volume,market,close_ratio,spread
0,bitcoin,BTC,Bitcoin,28-04-2013,1,108.240,108.784,105.680,107.368,0,1.500520e+09,0.5438,3.88
1,bitcoin,BTC,Bitcoin,29-04-2013,1,107.552,117.992,107.200,115.632,0,1.491160e+09,0.7813,13.49
2,bitcoin,BTC,Bitcoin,30-04-2013,1,115.200,117.544,107.240,111.200,0,1.597780e+09,0.3843,12.88
3,bitcoin,BTC,Bitcoin,01-05-2013,1,111.200,111.912,86.176,93.592,0,1.542820e+09,0.2882,32.17
4,bitcoin,BTC,Bitcoin,02-05-2013,1,93.104,100.480,73.824,84.168,0,1.292190e+09,0.3881,33.32


In [4]:
crypto_df.drop(labels=['slug', 'ranknow', 'volume', 'market', 'close_ratio', 'spread'], inplace=True, axis=1)
crypto_df.head()

,asset,name,date,open,high,low,close
0,BTC,Bitcoin,28-04-2013,108.240,108.784,105.680,107.368
1,BTC,Bitcoin,29-04-2013,107.552,117.992,107.200,115.632
2,BTC,Bitcoin,30-04-2013,115.200,117.544,107.240,111.200
3,BTC,Bitcoin,01-05-2013,111.200,111.912,86.176,93.592
4,BTC,Bitcoin,02-05-2013,93.104,100.480,73.824,84.168


In [13]:
#First, I created a sql connection, then check if any database with ‘Crypto’ name exists or not. 
#If it existed then I dropped it (Just for FUN :P). 
#And then created table Crypto with sql query inside conn.execute() method.
import sqlite3

# connect function opens a connection to the SQLite database file, 
conn = sqlite3.connect('session.db')
print(conn)
# Output: <sqlite3.Connection object at 0x0000015A87671730>

In [14]:
# Drop a table name Crypto if it exists already
try:
    conn.execute('DROP TABLE IF EXISTS `Crypto` ')
except Exception as e:
    raise(e)
finally:
    print('Table dropped')

    
# Create a new Table named as Crypto
try:
    conn.execute('''
         CREATE TABLE Crypto
         (ID         INTEGER PRIMARY KEY,
         ASSET       TEXT, 
         NAME        TEXT    NOT NULL,
         Date        datetime,
         Open        Float DEFAULT 0,
         High        Float DEFAULT 0,
         Low         Float DEFAULT 0,
         Close       Float DEFAULT 0);''')
    print ("Table created successfully");
except Exception as e:
    print(str(e))
    print('Table Creation Failed!!!!!')
finally:
    conn.close() # this closes the database connection

Table dropped
Table created successfully


In [15]:
#For data insertion, we again need to change our data from Pandas Dataframe to Python List of Lists or List of Tuples, 
#because that's the format sqlite module understand for data insertion. Below is the code snippet for same.

# this will convert pandas dtaframe to list of list
crypto_list = crypto_df.values.tolist()

# lets make new connection to Insert crypto data in SQL DB
conn = sqlite3.connect('session.db')

# make a cursor - it will help with querying SQL DB
cur = conn.cursor()

try:
    # will use ? sign to represent each column names inside VALUE().
    cur.executemany("INSERT INTO Crypto(ASSET, NAME, Date, Open, High, Low, Close) VALUES (?,?,?,?,?,?,?)", crypto_list)
    conn.commit()
    print('Data Inserted Successfully')
except Exception as e:
    print(str(e))
    print('Data Insertion Failed')
finally:
    # finally block will help with always closing the connection to DB even in case of error.
    conn.close()

# Output: Data Inserted Successfully


Data Inserted Successfully
